In [6]:
import pandas as pd
import os
import shutil
import librosa
import numpy as np
import soundfile as sf
from concurrent.futures import ThreadPoolExecutor
import tensorflow as tf
import random
import tensorflow_io as tfio
import preprocessing as pr
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import argparse as ap

In [7]:
!echo "Be sure to have tensorboard, this code will be commented in final release"
#!tensorboard --logdir ../../datasets/dsl_data/tensorboard_data/ &

"Be sure to have tensorboard, this code will be commented in final release"


 # Preprocessing for Train dataset files

In [8]:
df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')
new_folder_path = '../../datasets/dsl_data/Train_Dataset_Truncated/'

folder_path = '../../datasets/dsl_data/'

###############
def process_file(file_path):
    file_path_exists = df[df["path"] == file_path].shape[0] > 0 #flag
    # print(file_path)
    # print(file_path_exists)
    if file_path_exists:
        new_sr=16000
        # print(file_path)

        # identifier care
        identifier = df.loc[df["path"] == file_path, "Id"].values[0]
        identifier = str(int(identifier))
        # print("\n "+identifier)

        # label constructor

        action = df.loc[df["path"] == file_path, "action"].values[0]
        object = df.loc[df["path"] == file_path, "object"].values[0]
        label  = action + object

        new_file_path = os.path.join(new_folder_path, identifier + "_" + label + '.wav')

        # print(file_path)
        y, sr = librosa.load('../../datasets/'+file_path)
        # print("here")
        # print(y)
        y_truncated = librosa.effects.trim(y, top_db=50, frame_length=2048, hop_length=512, ref=np.max)[0]
        # print(y_truncated)
        y_truncated = librosa.resample(y_truncated, orig_sr=sr, target_sr=new_sr)

        y_truncated = y_truncated[:int(4*new_sr)] #if longer

        target_length = 4 * new_sr
        y_truncated = librosa.util.fix_length(data=y_truncated, size=target_length) #padding, if shorter

        sf.write(new_file_path, y_truncated, new_sr, 'PCM_16')

if not os.path.isdir(new_folder_path):
    os.makedirs(new_folder_path) #hoping to have write permissions set
if not os.listdir(new_folder_path):
    with ThreadPoolExecutor(max_workers=4) as executor: #multi-threaded beast :)
        for dirpath, dirnames, filenames in os.walk(folder_path):
            dirpath = dirpath.replace("\\", "/")
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            # print(dirpath)
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                file_path = file_path.replace("\\", "/")
                # print(file_path)
                # process_file(file_path)
                executor.submit(process_file, file_path)

# print(df)
print("Execution ended")


Execution ended


 # Preprocessing for Evaluation dataset files

In [9]:
df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')
new_folder_path = '../../datasets/dsl_data/Test_Dataset_Truncated/'

folder_path = '../../datasets/dsl_data/'

if not os.path.isdir(new_folder_path):
    os.makedirs(new_folder_path) #hoping to have write permissions set

def process_file(file_path):
    file_path_exists = df[df["path"] == file_path].shape[0] > 0 #flag
    # print(file_path_exists)
    if file_path_exists:
        new_sr=16000
        # print(file_path)

        # identifier care

        identifier = df.loc[df["path"] == file_path, "Id"].values[0]
        identifier = str(int(identifier))
        # print("\n " + identifier)

        # label constructor

        # action = df.loc[df["path"] == file_path, "action"].values[0]
        # object = df.loc[df["path"] == file_path, "object"].values[0]
        # label  = action + object

        new_file_path = os.path.join(new_folder_path, identifier + '.wav')

        y, sr = librosa.load('../../datasets/'+file_path)
        y_truncated = librosa.effects.trim(y, top_db=10, frame_length=2048, hop_length=512, ref=np.max)[0]

        y_truncated = librosa.resample(y_truncated, orig_sr=sr, target_sr=new_sr)

        y_truncated = y_truncated[:int(4*new_sr)]

        target_length = 4 * new_sr
        y_truncated = librosa.util.fix_length(data=y_truncated, size=target_length) #padding
        # print(new_file_path)
        sf.write(new_file_path, y_truncated, new_sr, 'PCM_16')

if not os.listdir(new_folder_path):
    with ThreadPoolExecutor(max_workers=4) as executor: #multi-threaded beast :)
        for dirpath, dirnames, filenames in os.walk(folder_path):
            dirpath = dirpath.replace("\\", "/")
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            dirpath = dirpath[dirpath.index("/")+1:]
            # print(dirpath)
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                file_path = file_path.replace("\\", "/")
                # print(file_path)
                executor.submit(process_file, file_path)

# print(df)
print("Execution ended")

Execution ended


# Auto - updating labels

In [10]:
df = pd.read_csv('../../datasets/dsl_data/development.csv', sep=',')

df['labels'] = df['action'].astype(str) + df['object'].astype(str)

distinct_values = df['labels'].unique()

# print(distinct_values)

result = 'LABELS = ['
for value in distinct_values:
    result += "'" + str(value) + "', "

result = result[:-2] + ']\n' # lazy workaround, the last label has a comma that is bad.. this is also bad.
# result += ']\n'

# Open the file in read mode
with open("preprocessing.py", "r") as file:
    # Read the contents of the file into a list
    lines = file.readlines()

# Update the 3rd line
lines[3] = result
lines[4] = "# This is the file genarated that has the Labels that i must use for training\n"

# Open the file in write mode
with open("preprocessing.py", "w") as file:
    # Write the updated lines back to the file
    file.writelines(lines)

# Model creation and fitting

In [11]:
parser = ap.ArgumentParser()

parser.add_argument('--batch_size', default=pr.TRAINING_ARGS['batch_size'], type=int, help="Choosing batch size default is 32")
parser.add_argument('--initial_learning_rate', default=pr.TRAINING_ARGS['initial_learning_rate'], type=float, help="Choosing initial_learning_rate")
parser.add_argument('--end_learning_rate', default=pr.TRAINING_ARGS['end_learning_rate'], type=float, help="Choosing end_learning_rate")
parser.add_argument('--epochs', default=pr.TRAINING_ARGS['epochs'], type=int, help="Choosing epochs")
parser.add_argument('--test_percentage', default=0.2, type=float, help="Choosing test_percentage")
parser.add_argument('--pruning_initial_step', default=0.2, type=float, help="Choosing pruning_initial_step")
parser.add_argument('--initial_sparsity', default=0.40, type=float, help="Choosing initial_sparsity")
parser.add_argument('--alpha', default=pr.alpha, type=float, help="Choosing alpha")

parser.add_argument('--eval_percentage', default=0.15, type=float, help="Choosing eval_percentage")

_StoreAction(option_strings=['--eval_percentage'], dest='eval_percentage', nargs=None, const=None, default=0.15, type=<class 'float'>, choices=None, required=False, help='Choosing eval_percentage', metavar=None)

Parser arguments

In [12]:
args = parser.parse_args(['--batch_size','32','--initial_learning_rate','0.03','--end_learning_rate','0.03','--epochs','5'])
# args = parser.parse_args()

In [13]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
random.seed(seed)
tf.random.set_seed(seed)
np.random.seed(seed)

# train_ds_location = './Train_Dataset_Truncated/'
# eval_ds_location  = './Test_Dataset_Truncated/'

train_ds_location   = '../../datasets/dsl_data/Train_Dataset_Truncated/'
#eval_ds_location    = '../../datasets/dsl_data/Test_Dataset_Truncated/'
eval_ds_location    = '../../datasets/dsl_data/Train_Dataset_Truncated/'

log_dir_tensorboard = '../../datasets/dsl_data/tensorboard_data/'
log_dir_model       = '../../datasets/dsl_data/models/'

runs = [int(d.split('_')[-1]) for d in os.listdir(log_dir_tensorboard) if 'run_' in d]
tb_run = max(runs) + 1 if runs else 0

Obtaining Test data from train data, using shuffle and avoiding retaking same data on different runs

In [14]:
file_paths = []

for filename in os.listdir(train_ds_location):
    file_path = os.path.join(train_ds_location, filename)
    file_paths.append(file_path)
random.shuffle(file_paths)
#test_percentage = 0.2
num_test_files = int(len(file_paths) * args.test_percentage)
num_eval_files = int(len(file_paths) * args.eval_percentage)

#args.eval_percentage
# it is shuffled, so i can do this
test_paths     = file_paths[:num_test_files]                 # from 0 to num_test_files
train_paths    = file_paths[num_test_files:-num_eval_files]  # from num_test_files to end-num_eval_files
eval_paths     = file_paths[-num_eval_files:]                # until the end


Preprocessing data and model creation

In [15]:
train_ds       = tf.data.Dataset.list_files(train_paths)
val_ds         = tf.data.Dataset.list_files(eval_ds_location)
test_ds        = tf.data.Dataset.list_files(test_paths)

#batch_size = pr.TRAINING_ARGS['batch_size']
#epochs = pr.TRAINING_ARGS['epochs']

train_ds       = train_ds.map(pr.preprocess).batch(args.batch_size).cache()
val_ds         = val_ds.map(pr.preprocess).batch(args.batch_size)
test_ds        = test_ds.map(pr.preprocess).batch(args.batch_size)

for example_batch, example_labels in train_ds.take(1):
  print('Batch Shape:', example_batch.shape)
  print('Data Shape:', example_batch.shape[1:])
  print('Labels:', example_labels)

prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
begin_step          = int(len(train_ds) * args.epochs * args.pruning_initial_step)
end_step            = int(len(train_ds) * args.epochs)
pruning_params      = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=args.initial_sparsity,
        final_sparsity=pr.final_sparsity,
        begin_step=begin_step,
        end_step=end_step
    )
}
custom_objects      = {'PruneLowMagnitude': prune_low_magnitude}

model_name          = 'model_'+str(args.batch_size)+'_'+str(args.alpha)+'.h5'

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=example_batch.shape[1:]),
    tf.keras.layers.Conv2D(filters=int(128 * args.alpha), kernel_size=[3, 3], strides=[2, 2],
        use_bias=False, padding='valid'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * args.alpha), kernel_size=[3, 3], strides=[1, 1],
            use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.Conv2D(filters=int(128 * args.alpha), kernel_size=[3, 3], strides=[1, 1],
        use_bias=False, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(units=len(pr.LABELS)),
    tf.keras.layers.Softmax()
    ])


model_for_pruning = prune_low_magnitude(model, **pruning_params)

Batch Shape: (32, 125, 31, 1)
Data Shape: (125, 31, 1)
Labels: tf.Tensor([5 1 4 3 4 3 4 4 3 1 5 3 2 6 2 3 4 5 0 4 1 3 4 3 3 3 3 6 4 6 1 1], shape=(32,), dtype=int64)


In [16]:
# this model uses Transfer Learning... I mean, we transferred a model developed for another course to this course
model_for_pruning.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_conv2d   (None, 62, 15, 128)      2306      
 (PruneLowMagnitude)                                             
                                                                 
 prune_low_magnitude_batch_n  (None, 62, 15, 128)      513       
 ormalization (PruneLowMagni                                     
 tude)                                                           
                                                                 
 prune_low_magnitude_re_lu (  (None, 62, 15, 128)      1         
 PruneLowMagnitude)                                              
                                                                 
 prune_low_magnitude_conv2d_  (None, 62, 15, 128)      294914    
 1 (PruneLowMagnitude)                                           
                                                        

In [17]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=False)
#initial_learning_rate = pr.TRAINING_ARGS['initial_learning_rate']
#end_learning_rate = pr.TRAINING_ARGS['end_learning_rate']

linear_decay = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=args.initial_learning_rate,
    end_learning_rate=args.end_learning_rate,
    decay_steps=len(train_ds) * args.epochs,
)
optimizer = tf.optimizers.Adam(learning_rate=linear_decay)
metrics = [tf.metrics.SparseCategoricalAccuracy()]

# callbacks = [tfmot.sparsity.keras.UpdatePruningStep()]
callbacks = [ tf.keras.callbacks.ModelCheckpoint(
    filepath=log_dir_model+model_name, 
    save_best_only=True, 
    save_weights_only=False, 
    monitor='val_loss', 
    mode='min', 
    save_freq='epoch'),
             tfmot.sparsity.keras.UpdatePruningStep(), 
             keras.callbacks.TensorBoard(log_dir=log_dir_tensorboard+'run_{}_epochs_{}_batch_size_{}_pruning_initial_step_{}_initial_learning_rate_{}_end_learning_rate_{}_test_percentage_{}_pruning_initial_step_{}_initial_sparsity_{}_alpha_{}'.format(tb_run,args.epochs,args.batch_size,args.pruning_initial_step,args.initial_learning_rate,args.end_learning_rate,args.test_percentage,args.pruning_initial_step,args.initial_sparsity,args.alpha), histogram_freq=1)]


model_for_pruning.compile(loss=loss, optimizer=optimizer, metrics=metrics)

history = model_for_pruning.fit(train_ds, epochs=args.epochs, validation_data=val_ds,callbacks=callbacks) #it was valds
# history = model_for_pruning.fit(train_ds, epochs=epochs, callbacks=callbacks) #it was valds


test_loss, test_accuracy = model_for_pruning.evaluate(test_ds)

training_loss = history.history['loss'][-1]
training_accuracy = history.history['sparse_categorical_accuracy'][-1]
val_loss = history.history['val_loss'][-1]
val_accuracy = history.history['val_sparse_categorical_accuracy'][-1]

print(f'Training Loss: {training_loss:.4f}')
print(f'Training Accuracy: {training_accuracy*100.:.2f}%')
print()
print(f'Validation Loss: {val_loss:.4f}')
print(f'Validation Accuracy: {val_accuracy*100.:.2f}%')
print()
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy*100.:.2f}%')

Epoch 1/5
201/201 [==============================] - ETA: 0s - loss: 1.8369 - sparse_categorical_accuracy: 0.2563

NotFoundError: Graph execution error:

NewRandomAccessFile failed to Create/Open: ..\..\datasets\dsl_data\Train_Dataset_Truncated\ : Impossibile trovare il percorso specificato.
; No such process
	 [[{{node ReadFile}}]]
	 [[IteratorGetNext]] [Op:__inference_test_function_6510]